### Task 1.1

## Calculate the Marginal Expected Shortfall (MES) of all banks
1. Time Period:
    * Year: 2019
2. Indian Bank
    * 12 Public Banks
    * 18 Private Banks

### Import Libraries

In [1]:
import os, sys, time, copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader as get_data
%matplotlib inline

### Define MES function

In [2]:
# Define slope of linear regression
def beta(x,y):
    x_bar = np.mean(x)
    y_bar = np.mean(y)
    S_xy = np.sum((x-x_bar)*(y-y_bar))
    S_xx = np.sum((x-x_bar)**2)
    return S_xy/S_xx
    
# Define MES measure (Minimum Expected Shortfall)
def mes(x,y):
    '''
    Input:
        x = individual returns
        y = Nifty50 returns
    ouput:
        -mean(returns of x at 5% worst Nifty returned cases)
    '''
    n = int(np.round(0.05*len(y),0))
    idx = np.argsort(y)[:n].tolist()
    required_arr = [x[i] for i in idx]
    return -np.mean(required_arr)

### Getting Information of Indian Banks

In [3]:
# load the bank details csv file
bank_details = pd.read_csv('./../data/bank_details.csv', encoding='latin')
# bank_details.dropna(subset = ['Symbol'], inplace = True)
bank_details.head()

,Sr. No,Name of the Bank,Branches,Establishment,Headquarter,Symbol,Type
0,1,Axis Bank,4528,1993,"Mumbai, Maharashtra",AXISBANK.NS,Private
1,2,Bandhan Bank,670+,2015,"Kolkata, West Bengal",BANDHANBNK.NS,Private
2,3,City Union Bank,700+,1904,"Kumbakonam, Tamil Nadu",CUB.NS,Private
3,4,D C B Bank,334,1930,"Mumbai, Maharashtra",DCBBANK.NS,Private
4,5,Dhanlaxmi Bank,270+,1927,"Thrissur city, Kerala",DHANBANK.NS,Private


In [4]:
# Collecting symbols for private and public banks
public_banks_symbol = bank_details[bank_details.Type == 'Public']['Symbol']
private_banks_symbol = bank_details[bank_details.Type == 'Private']['Symbol']
nifty50_symbol = '^NSEI'
print('Total number of public bank is ', len(public_banks_symbol))
print('Total number of private bank is ', len(private_banks_symbol))

Total number of public bank is  12
Total number of private bank is  18


### Define dates

In [5]:
# Fixing the date
start_date = '2019-01-01'
end_date = '2019-12-31'

### Nifty50

In [6]:
# Getting data for Nifty50 returns
nifty50 = get_data.DataReader(nifty50_symbol, 'yahoo', start = start_date, end = end_date, pause = 10)
nifty50.reset_index(drop = True, inplace = True)
nifty50 = nifty50['Close']
nifty50_return = nifty50.pct_change().drop(index = 0)

### Public Bank

In [7]:
# Collecting data for public banks
public_bank = get_data.DataReader(public_banks_symbol, 'yahoo', start = start_date, end = end_date, pause = 10)
public_bank = public_bank['Close']
public_bank.reset_index(drop = True, inplace = True)
public_bank_return = public_bank.pct_change().drop(index = 0)
public_bank_return.head()

Symbols,SBIN.NS,PNB.NS,BANKBARODA.BO,BANKINDIA.NS,MAHABANK.BO,UNIONBANK.NS,CANBK.NS,CENTRALBK.NS,INDIANB.NS,IOB.NS,PSB.NS,UCOBANK.NS
1,-0.019025,-0.024436,-0.031630,-0.014574,0.008409,-0.001133,-0.021670,0.001344,-0.021882,-0.006667,0.000000,-0.026432
2,-0.009527,0.000000,0.000000,0.009065,-0.030148,0.006236,0.003268,-0.008054,-0.001424,-0.023490,-0.013699,-0.045249
3,0.022501,0.044316,0.018007,0.027896,0.021164,0.039437,0.024068,-0.032476,0.017312,0.000000,-0.006173,0.030806
4,-0.004536,-0.009225,0.005348,-0.021159,-0.016839,-0.017886,-0.016611,-0.012587,-0.011211,0.003436,-0.004658,-0.022989
5,0.030375,0.014277,0.013093,0.022556,0.001976,0.043598,0.034681,0.012748,0.044139,0.013699,0.037442,0.011765


In [8]:
beta_coef = []
MES = []
for ticker in public_bank_return.columns.tolist():
    beta_coef.append(beta(nifty50_return, public_bank_return[ticker]))
    MES.append(mes(public_bank_return[ticker], nifty50_return))
public = {'Symbol':public_bank_return.columns.tolist(), 
            'Bank':bank_details[bank_details.Type == 'Public']['Name of the Bank'].tolist(),
            'Type':['Public' for i in range(public_bank_return.shape[1])], 'Beta':beta_coef,'MES':MES}
public = pd.DataFrame(public)
public

,Symbol,Bank,Type,Beta,MES
0,SBIN.NS,State bank of India,Public,0.247401,0.001617
1,PNB.NS,Punjab National Bank,Public,0.151553,0.005497
2,BANKBARODA.BO,Bank of Baroda,Public,0.180130,0.008463
3,BANKINDIA.NS,Bank of India,Public,0.208903,0.006856
4,MAHABANK.BO,Bank of Maharashtra,Public,-0.243627,0.001911
5,UNIONBANK.NS,Union Bank of India,Public,0.171471,0.009294
6,CANBK.NS,Canara Bank,Public,0.045707,0.006408
7,CENTRALBK.NS,Central Bank of India,Public,0.129819,0.005889
8,INDIANB.NS,Indian Bank,Public,-0.015461,0.019932
9,IOB.NS,Indian Overseas Bank,Public,-0.001133,0.002544


### Private Bank

In [9]:
# Collecting data for private banks
private_bank = get_data.DataReader(private_banks_symbol, 'yahoo', start = start_date, end = end_date, pause = 10)
private_bank = private_bank['Close']
private_bank.reset_index(drop = True, inplace = True)
private_bank_return = private_bank.pct_change().drop(index = 0)
private_bank_return.head()

Symbols,AXISBANK.NS,BANDHANBNK.NS,CUB.NS,DCBBANK.NS,DHANBANK.NS,FEDERALBNK.NS,HDFCBANK.NS,ICICIBANK.NS,IDBI.NS,IDFCFIRSTB.NS,INDUSINDBK.NS,J&KBANK.NS,KTKBANK.NS,KARURVYSYA.NS,KOTAKBANK.NS,RBLBANK.NS,SOUTHBANK.NS,YESBANK.NS
1,-0.011557,-0.008173,-0.003612,-0.003250,-0.005970,0.019355,-0.009124,0.002337,-0.001627,-0.014994,-0.011908,0.006477,-0.015011,0.014934,-0.007877,-0.020620,-0.012780,0.002171
2,-0.019515,-0.033234,-0.002072,-0.001482,0.000000,-0.029536,-0.007846,-0.003703,0.000000,0.008197,-0.010466,-0.003861,-0.012550,-0.003270,-0.004312,-0.005485,-0.009709,-0.002979
3,0.019163,-0.009179,0.011936,0.031760,-0.006006,0.030435,0.002675,0.005368,0.001630,0.048781,-0.007628,-0.023256,0.043577,-0.016949,0.010281,0.016990,0.000000,0.030147
4,0.028809,-0.058329,-0.025128,0.002014,-0.003021,-0.002110,0.001511,0.006846,-0.002441,0.022148,0.007073,-0.006614,-0.011744,0.023915,-0.001082,0.004373,-0.003268,-0.013182
5,0.021100,-0.042164,0.021305,0.018949,0.036364,0.006871,-0.008323,0.033859,0.009788,-0.003250,0.011930,-0.007989,0.018926,0.004889,-0.014199,-0.015937,0.000000,0.027785


In [10]:
beta_coef = []
MES = []
for ticker in private_bank_return.columns.tolist():
    beta_coef.append(beta(nifty50_return, private_bank_return[ticker]))
    MES.append(mes(private_bank_return[ticker], nifty50_return))
private = {'Symbol':private_bank_return.columns.tolist(), 
            'Bank':bank_details[bank_details.Type == 'Private']['Name of the Bank'].tolist(),
            'Type':['Private' for i in range(private_bank_return.shape[1])],'Beta':beta_coef, 'MES':MES}
private = pd.DataFrame(private)
private

,Symbol,Bank,Type,Beta,MES
0,AXISBANK.NS,Axis Bank,Private,0.187669,0.000529
1,BANDHANBNK.NS,Bandhan Bank,Private,0.253159,-0.003105
2,CUB.NS,City Union Bank,Private,0.084847,-0.002707
3,DCBBANK.NS,D C B Bank,Private,0.025086,0.006516
4,DHANBANK.NS,Dhanlaxmi Bank,Private,0.317355,0.008340
5,FEDERALBNK.NS,Federal Bank,Private,0.183374,0.005691
6,HDFCBANK.NS,H D F C Bank,Private,0.131439,-0.006533
7,ICICIBANK.NS,I C I C I Bank,Private,0.082753,-0.006333
8,IDBI.NS,I D B I Bank,Private,-0.030568,0.005766
9,IDFCFIRSTB.NS,I D F C FIRST Bank,Private,0.210198,0.003064


### Check vulnarability by median of MES

In [25]:
# Joint the both table
banks_vulnerability = pd.concat([public, private], ignore_index = True)
# Checking vulnerability by median value
banks_vulnerability['vulnerable'] = banks_vulnerability['MES'] < np.median(banks_vulnerability['MES'])
banks_vulnerability

,Symbol,Bank,Type,Beta,MES,vulnerable
0,SBIN.NS,State bank of India,Public,0.247401,0.001617,True
1,PNB.NS,Punjab National Bank,Public,0.151553,0.005497,False
2,BANKBARODA.BO,Bank of Baroda,Public,0.180130,0.008463,False
3,BANKINDIA.NS,Bank of India,Public,0.208903,0.006856,False
4,MAHABANK.BO,Bank of Maharashtra,Public,-0.243627,0.001911,True
5,UNIONBANK.NS,Union Bank of India,Public,0.171471,0.009294,False
6,CANBK.NS,Canara Bank,Public,0.045707,0.006408,False
7,CENTRALBK.NS,Central Bank of India,Public,0.129819,0.005889,False
8,INDIANB.NS,Indian Bank,Public,-0.015461,0.019932,False
9,IOB.NS,Indian Overseas Bank,Public,-0.001133,0.002544,True


In [26]:
# Save the file
banks_vulnerability.to_csv('./../data/banks_vulnerability.csv')